In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from TO_sim.Sol_Kuramoto import Sol_Kuramoto_mf2 as mf2
from TO_sim.Sol_Kuramoto import Sol_Kuramoto_mf2_sets as mf2_sets
from TO_sim.get_cluster import C_rsmso,cluster_os,C_rsmso_set_new
from TO_sim.To_Draw import Draw_avg_vel_r

from TO_sim.gen_Distribution import Normal
from TO_sim.gen_Distribution import Quantile_Normal as Q_Normal
import parmap
from scipy.signal import find_peaks
from TO_sim.To_Draw import Draw_avg_vel_r
from tqdm.notebook import tqdm


In [8]:
dK = 0.1
dm = 0.1
N = 500
seed = 11

theta_init,omega,Kc = Normal(N,0,1,seed=seed)
K_set = np.arange(0,5,dK).reshape(-1,1)

N_set = len(K_set)
omega = np.sort(omega) - np.mean(omega)
ms = np.arange(0,10+dm/2,dm)
m_set = []
core = 8
index_ = 0
for i in range(len(ms)//core +1):
    m_set.append(ms[index_:index_+core])
    index_ += core
core_set = [len(ms) for ms in m_set]
N_set = len(K_set)
theta_init_set = (np.ones((N_set,1))*theta_init)
pdtheta_set = 0*(np.ones((N_set,1))*omega)
omega_set = (np.ones((N_set,1))*omega)

In [9]:
core_set

[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 5]

In [10]:
cols ={'r':[0],'rstd':[0],'rMM':[0],'error':[0]}
for i in range(10):
    cols[f'c{i}'] = np.nan
for i in range(10):
    cols[f'c{i} phase vel'] = np.nan
for i in range(10):
    cols[f'c{i} omega'] = np.nan

cols['K'] = K_set.reshape(-1)[0]
cols['m'] = ms[0]
df = pd.DataFrame(cols)
df_Km = df.set_index(['K','m'])

In [14]:
for core,ms in tqdm(list(zip(core_set,m_set))):pass

  0%|          | 0/13 [00:00<?, ?it/s]

In [15]:
for core,ms in zip(core_set,m_set):
    for m,(r,rstd,rMM,CSMP_set,CSMO_set,error_set,C_s_set,C_e_set) in zip(ms,parmap.map(C_rsmso_set_new,ms,K_set=K_set,N=N,theta_init_set=theta_init_set,
                                                        omega_set=omega_set,pdtheta_set = pdtheta_set,t_end=2000,dt=0.1,
                                                        pm_pbar=True, pm_processes=core)):
        for i,K in enumerate(K_set.reshape(-1)):
            df_Km.loc[(K,m),"r"] = r[i]
            df_Km.loc[(K,m),"rstd"] = rstd[i]
            df_Km.loc[(K,m),"rMM"] = rMM[i]
            df_Km.loc[(K,m),'error'] = error_set[i]
            CSMP = CSMP_set[i]
            CSMO = CSMO_set[i]
            for i in range(len(CSMP_set[i])):
                    df_Km.loc[(K,m),f'c{i}'] = CSMP['cluster size'][i]
                    df_Km.loc[(K,m),f'c{i} phase vel'] = CSMP['cluster mean phase velocity'][i]
                    df_Km.loc[(K,m),f'c{i} omega'] = CSMO['cluster mean natural frequency'][i]

  0%|          | 0/8 [00:00<?, ?it/s]

/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster

  0%|          | 0/8 [00:00<?, ?it/s]

/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster

  0%|          | 0/8 [00:00<?, ?it/s]

...

/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster

  0%|          | 0/8 [00:00<?, ?it/s]

/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster

  0%|          | 0/5 [00:00<?, ?it/s]

/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster

In [16]:
K_set = np.arange(5,10+dK/2,dK).reshape(-1,1)
N_set = len(K_set)
omega = np.sort(omega) - np.mean(omega)
ms = np.arange(0,10+dm/2,dm)
m_set = []
core = 8
index_ = 0
for i in range(len(ms)//core +1):
    m_set.append(ms[index_:index_+core])
    index_ += core
core_set = [len(ms) for ms in m_set]
N_set = len(K_set)
theta_init_set = (np.ones((N_set,1))*theta_init)
pdtheta_set = 0*(np.ones((N_set,1))*omega)
omega_set = (np.ones((N_set,1))*omega)
for core,ms in zip(core_set,m_set):
    for m,(r,rstd,rMM,CSMP_set,CSMO_set,error_set,C_s_set,C_e_set) in zip(ms,parmap.map(C_rsmso_set_new,ms,K_set=K_set,N=N,theta_init_set=theta_init_set,
                                                        omega_set=omega_set,pdtheta_set = pdtheta_set,t_end=2000,dt=0.1,
                                                        pm_pbar=True, pm_processes=core)):
        for i,K in enumerate(K_set.reshape(-1)):
            df_Km.loc[(K,m),"r"] = r[i]
            df_Km.loc[(K,m),"rstd"] = rstd[i]
            df_Km.loc[(K,m),"rMM"] = rMM[i]
            df_Km.loc[(K,m),'error'] = error_set[i]
            CSMP = CSMP_set[i]
            CSMO = CSMO_set[i]
            for i in range(len(CSMP_set[i])):
                    df_Km.loc[(K,m),f'c{i}'] = CSMP['cluster size'][i]
                    df_Km.loc[(K,m),f'c{i} phase vel'] = CSMP['cluster mean phase velocity'][i]
                    df_Km.loc[(K,m),f'c{i} omega'] = CSMO['cluster mean natural frequency'][i]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster

...

/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster

  0%|          | 0/8 [00:00<?, ?it/s]

/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster

  0%|          | 0/5 [00:00<?, ?it/s]

/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster

In [17]:
df_Km.sort_index().to_excel(f'N = {N} seed = {seed} test 230708 ver3.xlsm')

In [18]:
for seed in [12,13,14]:
    dK = 0.1
    dm = 0.1
    N = 500

    theta_init,omega,Kc = Normal(N,0,1,seed=seed)
    K_set = np.arange(0,5,dK).reshape(-1,1)

    N_set = len(K_set)
    omega = np.sort(omega) - np.mean(omega)
    ms = np.arange(0,10+dm/2,dm)
    m_set = []
    core = 8
    index_ = 0
    for i in range(len(ms)//core +1):
        m_set.append(ms[index_:index_+core])
        index_ += core
    core_set = [len(ms) for ms in m_set]
    N_set = len(K_set)
    theta_init_set = (np.ones((N_set,1))*theta_init)
    pdtheta_set = 0*(np.ones((N_set,1))*omega)
    omega_set = (np.ones((N_set,1))*omega)
    core_set
    cols ={'r':[0],'rstd':[0],'rMM':[0],'error':[0]}
    for i in range(10):
        cols[f'c{i}'] = np.nan
    for i in range(10):
        cols[f'c{i} phase vel'] = np.nan
    for i in range(10):
        cols[f'c{i} omega'] = np.nan

    cols['K'] = K_set.reshape(-1)[0]
    cols['m'] = ms[0]
    df = pd.DataFrame(cols)
    df_Km = df.set_index(['K','m'])
    for core,ms in tqdm(list(zip(core_set,m_set))):pass
    for core,ms in zip(core_set,m_set):
        for m,(r,rstd,rMM,CSMP_set,CSMO_set,error_set,C_s_set,C_e_set) in zip(ms,parmap.map(C_rsmso_set_new,ms,K_set=K_set,N=N,theta_init_set=theta_init_set,
                                                            omega_set=omega_set,pdtheta_set = pdtheta_set,t_end=2000,dt=0.1,
                                                            pm_pbar=True, pm_processes=core)):
            for i,K in enumerate(K_set.reshape(-1)):
                df_Km.loc[(K,m),"r"] = r[i]
                df_Km.loc[(K,m),"rstd"] = rstd[i]
                df_Km.loc[(K,m),"rMM"] = rMM[i]
                df_Km.loc[(K,m),'error'] = error_set[i]
                CSMP = CSMP_set[i]
                CSMO = CSMO_set[i]
                for i in range(len(CSMP_set[i])):
                        df_Km.loc[(K,m),f'c{i}'] = CSMP['cluster size'][i]
                        df_Km.loc[(K,m),f'c{i} phase vel'] = CSMP['cluster mean phase velocity'][i]
                        df_Km.loc[(K,m),f'c{i} omega'] = CSMO['cluster mean natural frequency'][i]
    K_set = np.arange(5,10+dK/2,dK).reshape(-1,1)
    N_set = len(K_set)
    omega = np.sort(omega) - np.mean(omega)
    ms = np.arange(0,10+dm/2,dm)
    m_set = []
    core = 8
    index_ = 0
    for i in range(len(ms)//core +1):
        m_set.append(ms[index_:index_+core])
        index_ += core
    core_set = [len(ms) for ms in m_set]
    N_set = len(K_set)
    theta_init_set = (np.ones((N_set,1))*theta_init)
    pdtheta_set = 0*(np.ones((N_set,1))*omega)
    omega_set = (np.ones((N_set,1))*omega)
    for core,ms in zip(core_set,m_set):
        for m,(r,rstd,rMM,CSMP_set,CSMO_set,error_set,C_s_set,C_e_set) in zip(ms,parmap.map(C_rsmso_set_new,ms,K_set=K_set,N=N,theta_init_set=theta_init_set,
                                                            omega_set=omega_set,pdtheta_set = pdtheta_set,t_end=2000,dt=0.1,
                                                            pm_pbar=True, pm_processes=core)):
            for i,K in enumerate(K_set.reshape(-1)):
                df_Km.loc[(K,m),"r"] = r[i]
                df_Km.loc[(K,m),"rstd"] = rstd[i]
                df_Km.loc[(K,m),"rMM"] = rMM[i]
                df_Km.loc[(K,m),'error'] = error_set[i]
                CSMP = CSMP_set[i]
                CSMO = CSMO_set[i]
                for i in range(len(CSMP_set[i])):
                        df_Km.loc[(K,m),f'c{i}'] = CSMP['cluster size'][i]
                        df_Km.loc[(K,m),f'c{i} phase vel'] = CSMP['cluster mean phase velocity'][i]
                        df_Km.loc[(K,m),f'c{i} omega'] = CSMO['cluster mean natural frequency'][i]
    df_Km.sort_index().to_excel(f'N = {N} seed = {seed} test 230708 ver3.xlsm')


  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster

  0%|          | 0/8 [00:00<?, ?it/s]

/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster

...

/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster

  0%|          | 0/8 [00:00<?, ?it/s]

/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster

  0%|          | 0/5 [00:00<?, ?it/s]

/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster.py:278: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cluster = np.array([np.arange(c_i,c_j,1) for c_i,c_j in zip(C_start[Is_group,-1],C_end[Is_group,-1])])
/Users/kimgug-young/Desktop/KURAMOTO 1st order/Kuramoto-model/TO_sim/get_cluster